In [5]:
# -*- coding: utf-8 -*-
"""Neuro-Fuzzy System Example"""

#!pip install scikit-fuzzy # Uncomment and run this line if you don't have scikit-fuzzy installed

import numpy as np
import random # Although random is imported, np.random is primarily used
import matplotlib.pyplot as plt # Imported but not explicitly used in the provided snippets

import skfuzzy as fuzz
from skfuzzy import control as ctrl

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# --- Step 1: Fuzzy Logic Setup ---
# Define Fuzzy Antecedent (Input) Variables and their Universes of Discourse

# Speed: Range from 0 to 150 (e.g., km/h or mph)
speed = ctrl.Antecedent(np.arange(0, 151, 1), 'speed')

# Weather: Categorical or scaled numerical input (e.g., 0=clear, 1=rainy, 2=foggy)
# Using a universe from 0 to 3 for simplicity based on the data generation approach
weather = ctrl.Antecedent(np.arange(0, 3, 1), 'weather')

# Define Membership Functions for each Antecedent Variable
# Membership functions define how crisp input values map to fuzzy sets (e.g., 'slow', 'medium').

# Membership functions for speed
speed['slow'] = fuzz.trimf(speed.universe, [0, 0, 60])      # Triangular membership function
speed['medium'] = fuzz.trimf(speed.universe, [40, 80, 120]) # Triangular
speed['fast'] = fuzz.trimf(speed.universe, [100, 150, 150]) # Triangular

# Membership functions for weather
# These define the degree to which a weather value corresponds to 'clear', 'rainy', or 'foggy'.
# Note: The universe is 0-3, but data is generated as 0, 1, 2.
# The triangular functions map these crisp values to membership degrees.
weather['clear'] = fuzz.trimf(weather.universe, [0, 0, 0.5])   # Peak at 0 (clear)
weather['rainy'] = fuzz.trimf(weather.universe, [0.5, 1, 1.5]) # Peak at 1 (rainy)
weather['foggy'] = fuzz.trimf(weather.universe, [1.5, 2, 2]) # Peak at 2 (foggy)

# Function to fuzzify crisp inputs into membership degrees
def fuzzify_inputs(speed_val, weather_val):
    """
    Takes crisp speed and weather values and returns a list of their
    membership degrees for each fuzzy set. These serve as fuzzy features.
    """
    # Calculate membership degrees for speed fuzzy sets
    speed_lvls = [
        fuzz.interp_membership(speed.universe, speed['slow'].mf, speed_val),
        fuzz.interp_membership(speed.universe, speed['medium'].mf, speed_val),
        fuzz.interp_membership(speed.universe, speed['fast'].mf, speed_val)
    ]

    # Calculate membership degrees for weather fuzzy sets
    weather_lvls = [
        fuzz.interp_membership(weather.universe, weather['clear'].mf, weather_val),
        fuzz.interp_membership(weather.universe, weather['rainy'].mf, weather_val),
        fuzz.interp_membership(weather.universe, weather['foggy'].mf, weather_val)
    ]

    # Concatenate the membership degrees from both inputs
    # This forms a single feature vector (fuzzy features) for the neural network
    return speed_lvls + weather_lvls

# --- Step 2: Generate Synthetic Dataset ---
# Create a dataset to train the neural network.
# The inputs are speed and weather. The output (label) is 'Low', 'Medium', or 'High' risk,
# determined by a simple rule-based system acting as the ground truth.

np.random.seed(0) # Seed for reproducibility
X_raw = [] # To store the fuzzy features (inputs for NN)
y_raw = [] # To store the crisp risk labels (outputs for NN training)

for _ in range(1000): # Generate 1000 data samples
    # Generate random crisp inputs for speed and weather
    spd = np.random.uniform(0, 150) # Random speed between 0 and 150
    wthr = np.random.choice([0, 1, 2]) # Random weather: 0 (clear), 1 (rainy), or 2 (foggy)

    # Fuzzify the crisp inputs to get the fuzzy features
    fuzzy_features = fuzzify_inputs(spd, wthr)
    X_raw.append(fuzzy_features) # Add fuzzy features to input list

    # Rule-based label generation (Simulating a simplified expert system or ground truth)
    # This defines what the neural network should learn to predict.
    if spd > 100 and wthr > 0: # If speed is high AND weather is not clear (rainy or foggy)
        y_raw.append('High')
    elif spd > 50: # If speed is moderate to high (and not already classified as High)
        y_raw.append('Medium')
    else: # Otherwise (speed is low or moderate in clear weather)
        y_raw.append('Low')

# Encode labels: Convert the string labels ('Low', 'Medium', 'High') into numerical format
# that the neural network can understand (e.g., 0, 1, 2).
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_raw) # Fit and transform the raw labels

# Convert lists to NumPy arrays for use with TensorFlow/Keras and scikit-learn
X = np.array(X_raw) # Features for the neural network
y = np.array(y_encoded) # Numerical labels for the neural network

# --- Step 3: Train Neural Network ---
# Use the generated dataset (fuzzy features X and encoded labels y) to train a simple
# feedforward neural network. The network learns the mapping from fuzzy input states
# to risk categories based on the synthetic ground truth rules.

# Split the dataset into training and testing sets
# train_test_split shuffles the data and splits it into 80% for training and 20% for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the Neural Network Model (Sequential API)
# This is a simple feedforward network (Multi-Layer Perceptron - MLP).
model = Sequential([
    # Input layer and first hidden layer:
    # input_dim=6 because fuzzify_inputs returns a list of 6 values (3 for speed, 3 for weather).
    # Dense is a fully connected layer. 12 is the number of neurons in this layer.
    # 'relu' (Rectified Linear Unit) is a common activation function.
    Dense(12, input_dim=6, activation='relu'),

    # Second hidden layer: 8 neurons with 'relu' activation.
    Dense(8, activation='relu'),

    # Output layer:
    # 3 neurons, corresponding to the 3 risk classes ('Low', 'Medium', 'High').
    # 'softmax' activation outputs a probability distribution over the classes,
    # summing to 1. The class with the highest probability is the predicted class.
    Dense(3, activation='softmax') # 3 classes: Low, Medium, High
])

# Compile the Model: Configure the model for training
# loss='sparse_categorical_crossentropy': Suitable for multi-class classification with integer labels.
# optimizer='adam': A popular and effective optimization algorithm.
# metrics=['accuracy']: Monitor the accuracy during training.
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the Model: Fit the model to the training data
# epochs=50: The number of times the entire training dataset is passed forward and backward through the network.
# batch_size=16: The number of samples per gradient update.
# verbose=1: Display training progress.
history = model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=1)

print("\nNeural Network Training Finished.")

# --- Step 4: Evaluate Neural Network ---
# Assess the performance of the trained network on the unseen test data.
print("\nEvaluating the trained Neural Network on the test set...")
loss, accuracy = model.evaluate(X_test, y_test, verbose=0) # verbose=0 hides progress bar

# Print the evaluation result
print(f"Test Accuracy: {accuracy:.4f}")

# --- Example Prediction using the combined system ---
# This function demonstrates how a real-world input (crisp speed and weather)
# goes through the fuzzy logic part (fuzzification) and then the neural network
# for final classification.
def predict_risk(speed_input, weather_input):
    """
    Predicts the risk level for given crisp speed and weather inputs
    using the combined Fuzzy Logic and Neural Network system.
    """
    # 1. Fuzzify the crisp inputs to get fuzzy features
    fuzzy_input = np.array(fuzzify_inputs(speed_input, weather_input)).reshape(1, -1)
    # .reshape(1, -1) ensures the input is in the correct shape (1 sample, 6 features)
    # expected by the neural network.

    # 2. Feed the fuzzy features into the trained Neural Network for prediction
    prediction = model.predict(fuzzy_input)

    # 3. Get the predicted class index (the one with the highest probability)
    predicted_class_index = np.argmax(prediction)

    # 4. Convert the predicted class index back to the original string label
    predicted_label = label_encoder.inverse_transform([predicted_class_index])[0]

    return predicted_label

# --- Try predicting with example inputs ---
print("\nTesting the combined system with example inputs:")
# Example 1: Fast speed (120) and Rainy weather (1)
predicted_risk_1 = predict_risk(120, 1)
print(f"Predicted Risk for Speed=120, Weather=1 (Rainy): {predicted_risk_1}") # Expected: High

# Example 2: Slow speed (30) and Clear weather (0)
predicted_risk_2 = predict_risk(30, 0)
print(f"Predicted Risk for Speed=30, Weather=0 (Clear): {predicted_risk_2}") # Expected: Low

# Example 3: Moderate speed (70) and Foggy weather (2)
predicted_risk_3 = predict_risk(70, 2)
print(f"Predicted Risk for Speed=70, Weather=2 (Foggy): {predicted_risk_3}") # Expected: Medium or High depending on exact position in fuzzy sets

c:\Users\Pranav\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.3694 - loss: 1.0781
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5221 - loss: 0.9933
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6228 - loss: 0.9164
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7335 - loss: 0.8237
Epoch 5/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8570 - loss: 0.6888
Epoch 6/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9195 - loss: 0.5397
Epoch 7/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9419 - loss: 0.4275
Epoch 8/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9387 - loss: 0.3434
Epoch 9/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9434 - loss: 0.2760
Epoch 10/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9514 - loss: 0.2141
Epoch 11/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9526 - loss: 0.1907
Epoch 12/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9589 - lo

In [4]:

import sys
!{sys.executable} -m pip install tensorflow
